In [2]:
!conda install -c anaconda beautifulsoup4 --yes
# import libraries
from bs4 import BeautifulSoup
import urllib.request
import csv
import pandas as pd

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |           py36_0         165 KB  anaconda
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    certifi-2020.4.5.1         |           py36_0         159 KB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    soupsieve-2.0              |             py_0          33 KB  anaconda
    ------------------------------------------------------------
                                           Total:         4.3 MB

The following NEW packages will be INSTALLED:

  beautifulsoup4     anaconda/linux-64::beautifulsoup4-4.9.0-py36_0
  soupsieve          anaconda/noarch::soupsieve-2.0-py_0

The following packages

In [3]:
# specify the url
urlpage = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# query the website and return the html to the variable 'page'
page = urllib.request.urlopen(urlpage)
# parse the html using beautiful soup and store in variable 
bsObj = BeautifulSoup(page, 'html.parser')
# find results within table
tab = bsObj.findAll("tbody")
data_list=[]
for ligne in tab[0].findAll("tr"):
    l=[]
    for cellule in ligne.findAll("td"):
        l.append(((str(cellule).replace("<td>","")).replace("</td>","")).replace("\n",""))
    data_list.append(l)
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
data=pd.DataFrame(data=data_list,columns=['Postalcode','Borough','Neighborhood'])
data.drop([0], inplace=True)

In [4]:
#Ignore cells with a borough that is Not assigned.
data.drop(data[data.Borough =='Not assigned'].index, inplace=True)

In [5]:
#More than one neighborhood can exist in one postal code area.
data['Neighborhood'] = data.Neighborhood.str.replace('/',',')

In [6]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
for row in data.iterrows():
    if row[1]['Neighborhood']=="":
        row[1]['Neighborhood']=row[1]['Borough']

In [7]:
data  

,Postalcode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park , Harbourfront"
6,M6A,North York,"Lawrence Manor , Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
...,...,...,...
161,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
166,M4Y,Downtown Toronto,Church and Wellesley
169,M7Y,East Toronto,Business reply mail Processing Centre
170,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


In [8]:
data.shape

(103, 3)

In [9]:
geo_data=pd.read_csv('Geospatial_Coordinates.csv')
geo_data.rename(columns={"Postal Code": 'Postalcode'}, inplace=True)
geo_data

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [10]:
data2= data.assign(Latitude="", Longitude="")

In [11]:
#import geocoder # import geocoder
import numpy as np
for row in data2.iterrows():
    p=row[1]['Postalcode']
    x=geo_data[geo_data['Postalcode']==p][['Latitude','Longitude']].values[0]
    if x == []:
        print(p)
    else:
        row[1]['Latitude']=x[0]
        row[1]['Longitude']=x[1]
data2  

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


,Postalcode,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.7533,-79.3297
4,M4A,North York,Victoria Village,43.7259,-79.3156
5,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6543,-79.3606
6,M6A,North York,"Lawrence Manor , Lawrence Heights",43.7185,-79.4648
7,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6623,-79.3895
...,...,...,...,...,...
161,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.6537,-79.5069
166,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
169,M7Y,East Toronto,Business reply mail Processing Centre,43.6627,-79.3216
170,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.6363,-79.4985


In [51]:
#we will work with only boroughs that contain the word Toronto
def verif(row):
    return('Toronto' in row['Borough'])
data2=data2[data2.apply(verif, axis=1)]
print(len(data2['Postalcode'].unique()))
data2

39


,Postalcode,Borough,Neighborhood,Latitude,Longitude
5,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6543,-79.3606
7,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6623,-79.3895
14,M5B,Downtown Toronto,"Garden District , Ryerson",43.6572,-79.3789
23,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
31,M4E,East Toronto,The Beaches,43.6764,-79.293
32,M5E,Downtown Toronto,Berczy Park,43.6448,-79.3733
41,M5G,Downtown Toronto,Central Bay Street,43.658,-79.3874
42,M6G,Downtown Toronto,Christie,43.6695,-79.4226
50,M5H,Downtown Toronto,"Richmond , Adelaide , King",43.6506,-79.3846
51,M6H,West Toronto,"Dufferin , Dovercourt Village",43.669,-79.4423


In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    CLIENT_ID = '53EEVY3F5MW5S2I2FV4YGU4QTBA3B41DBJDAA1PDDNWRBEPP' # your Foursquare ID
    CLIENT_SECRET = 'ICGTZOZHWLF0ATDISYNA0HM4SO0EUUAGE5HUZZ0FEKPA1GT2' # your Foursquare Secret
    VERSION = '20180605' # Foursquare API version
    LIMIT=100
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [46]:

Toronto_venues = getNearbyVenues(names=data2['Neighborhood'],
                                   latitudes=data2['Latitude'],
                                   longitudes=data2['Longitude']
                                  )

Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [47]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton , Parkdale Village , Exhibition Place",23,23,23,23,23,23
Business reply mail Processing Centre,16,16,16,16,16,16
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",16,16,16,16,16,16
Central Bay Street,64,64,64,64,64,64
Christie,17,17,17,17,17,17
Church and Wellesley,78,78,78,78,78,78
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
Davisville,32,32,32,32,32,32


In [53]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 231 uniques categories.


In [54]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
#group rows by neighborhood and by taking the mean of the frequency
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.043478,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing Centre,0.062500,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.015625,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00,0.015625,0.000000,0.000000,0.000000,0.000000
5,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.025641,0.012821,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012821,...,0.012821,0.012821,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court , Victoria Hotel",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,...,0.000000,0.000000,0.00000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
8,Davisville,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.03125,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [56]:
Toronto_grouped.shape

(39, 231)

In [57]:
#Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2                Café  0.04
3  Seafood Restaurant  0.04
4         Cheese Shop  0.04


----Brockton , Parkdale Village , Exhibition Place----
            venue  freq
0            Café  0.13
1     Coffee Shop  0.09
2  Breakfast Spot  0.09
3     Yoga Studio  0.04
4    Intersection  0.04


----Business reply mail Processing Centre----
                venue  freq
0         Yoga Studio  0.06
1       Auto Workshop  0.06
2       Garden Center  0.06
3              Garden  0.06
4  Light Rail Station  0.06


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3          Boutique  0.06
4             Plane  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1                Café  0.08
2  

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [59]:
#let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Café,Cheese Shop,Restaurant,Bistro,Breakfast Spot
1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Breakfast Spot,Yoga Studio,Climbing Gym,Stadium,Bakery,Intersection,Burrito Place,Italian Restaurant
2,Business reply mail Processing Centre,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Butcher,Burrito Place,Restaurant,Brewery,Light Rail Station
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Boutique,Rental Car Location,Bar,Plane
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Thai Restaurant,Bubble Tea Shop,Bar,Burger Joint,Dessert Shop,Japanese Restaurant


In [65]:
#Cluster Neighborhoods

# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 4, 0, 1, 0,
       0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [68]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = data2

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6543,-79.3606,0,Coffee Shop,Bakery,Park,Pub,Theater,Restaurant,Café,Breakfast Spot,Yoga Studio,Performing Arts Venue
7,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6623,-79.3895,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Discount Store,Juice Bar,Bank,Bar,Café,Diner,Burrito Place
14,M5B,Downtown Toronto,"Garden District , Ryerson",43.6572,-79.3789,0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Restaurant,Middle Eastern Restaurant,Hotel
23,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754,0,Café,Coffee Shop,American Restaurant,Gastropub,Cocktail Bar,Clothing Store,Beer Bar,Restaurant,Gym,Italian Restaurant
31,M4E,East Toronto,The Beaches,43.6764,-79.293,0,Coffee Shop,Trail,Health Food Store,Pub,College Rec Center,Colombian Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [73]:
#Finally, let's visualize the resulting clusters
# create map
latitude=43.6764
longitude=-79.293

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters